In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# https://drive.google.com/file/d/1jU38cWxHrKGLg1nWO_wW8031z7d1HpnO/view?usp=drive_link

id='1jU38cWxHrKGLg1nWO_wW8031z7d1HpnO'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('googleplaystore.csv')

In [ ]:
# https://drive.google.com/file/d/1wJgpI28LxgXlWxeLfTJE2O89l3dPbekL/view?usp=drive_link

id='1wJgpI28LxgXlWxeLfTJE2O89l3dPbekL'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('googleplaystoreuserreviews.csv')

Abrimos el archivo cargando unicamente las columnas necesarias para este ejercicio. La columna Rating no la necesitamos, pero sabemos de otros
ejercicios que hay una fila con un rating imposible, por lo que esa fila hay que eliminarla

In [ ]:
columnas_a_usar1 = ["App", "Rating", "Category", "Last Updated"]
apps = pd.read_csv("googleplaystore.csv", usecols = columnas_a_usar1)
columnas_a_usar2 = ["App","Sentiment_Polarity", "Sentiment_Subjectivity"]
appsreviews = pd.read_csv("googleplaystoreuserreviews.csv", usecols = columnas_a_usar2)

In [ ]:
apps

,App,Category,Rating,Last Updated
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,"January 7, 2018"
1,Coloring book moana,ART_AND_DESIGN,3.9,"January 15, 2018"
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,"August 1, 2018"
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,"June 8, 2018"
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,"June 20, 2018"
...,...,...,...,...
10836,Sya9a Maroc - FR,FAMILY,4.5,"July 25, 2017"
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,"July 6, 2018"
10838,Parkinson Exercices FR,MEDICAL,NaN,"January 20, 2017"
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,"January 19, 2015"


In [ ]:
apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   App           10841 non-null  object 
 1   Category      10841 non-null  object 
 2   Rating        9367 non-null   float64
 3   Last Updated  10841 non-null  object 
dtypes: float64(1), object(3)
memory usage: 338.9+ KB


Hay aplicaciones repetidas, en mi criterio las sacamos ya que una misma app no puede tener datos diferentes.

In [ ]:
apps["App"].duplicated().sum()

1181

Nos quedamos unicamente con la primer repeticion de aquellas Apps que se repiten

In [ ]:
apps1 = apps.drop_duplicates(subset = "App")
apps1

,App,Category,Rating,Last Updated
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,"January 7, 2018"
1,Coloring book moana,ART_AND_DESIGN,3.9,"January 15, 2018"
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,"August 1, 2018"
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,"June 8, 2018"
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,"June 20, 2018"
...,...,...,...,...
10836,Sya9a Maroc - FR,FAMILY,4.5,"July 25, 2017"
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,"July 6, 2018"
10838,Parkinson Exercices FR,MEDICAL,NaN,"January 20, 2017"
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,"January 19, 2015"


Los eliminamos correctamente

In [ ]:
apps1["App"].duplicated().sum()

0

In [ ]:
apps1["Rating"].unique()

array([ 4.1,  3.9,  4.7,  4.5,  4.3,  4.4,  3.8,  4.2,  4.6,  3.2,  4. ,
        nan,  4.8,  4.9,  3.6,  3.7,  3.3,  3.4,  3.5,  3.1,  5. ,  2.6,
        3. ,  1.9,  2.5,  2.8,  2.7,  1. ,  2.9,  2.3,  2.2,  1.7,  2. ,
        1.8,  2.4,  1.6,  2.1,  1.4,  1.5,  1.2, 19. ])

Hay un rating que no esta entre 1 y 5, lo eliminamos. Los ratings nans en este ejercicio los dejamos ya que no trabajamos mas con esta columna

In [ ]:
apps2 = apps1.drop(apps1[apps1["Rating"] > 5].index)
apps2["Rating"].unique()

array([4.1, 3.9, 4.7, 4.5, 4.3, 4.4, 3.8, 4.2, 4.6, 3.2, 4. , nan, 4.8,
       4.9, 3.6, 3.7, 3.3, 3.4, 3.5, 3.1, 5. , 2.6, 3. , 1.9, 2.5, 2.8,
       2.7, 1. , 2.9, 2.3, 2.2, 1.7, 2. , 1.8, 2.4, 1.6, 2.1, 1.4, 1.5,
       1.2])

Vemos las categorias del df

In [ ]:
apps2["Category"].unique()

array(['ART_AND_DESIGN', 'AUTO_AND_VEHICLES', 'BEAUTY',
       'BOOKS_AND_REFERENCE', 'BUSINESS', 'COMICS', 'COMMUNICATION',
       'DATING', 'EDUCATION', 'ENTERTAINMENT', 'EVENTS', 'FINANCE',
       'FOOD_AND_DRINK', 'HEALTH_AND_FITNESS', 'HOUSE_AND_HOME',
       'LIBRARIES_AND_DEMO', 'LIFESTYLE', 'GAME', 'FAMILY', 'MEDICAL',
       'SOCIAL', 'SHOPPING', 'PHOTOGRAPHY', 'SPORTS', 'TRAVEL_AND_LOCAL',
       'TOOLS', 'PERSONALIZATION', 'PRODUCTIVITY', 'PARENTING', 'WEATHER',
       'VIDEO_PLAYERS', 'NEWS_AND_MAGAZINES', 'MAPS_AND_NAVIGATION'],
      dtype=object)

Vemos el formato de las fechas

In [ ]:
apps2["Last Updated"].unique()

array(['January 7, 2018', 'January 15, 2018', 'August 1, 2018', ...,
       'January 20, 2014', 'February 16, 2014', 'March 23, 2014'],
      dtype=object)

Primero convertimos los strings de la columna Last Updated a formato datetime

In [ ]:
apps3 = apps2.copy()
apps3["Last Updated"] = pd.to_datetime(apps3["Last Updated"], format = "%B %d, %Y")
apps3

,App,Category,Rating,Last Updated
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,2018-01-07
1,Coloring book moana,ART_AND_DESIGN,3.9,2018-01-15
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,2018-08-01
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,2018-06-08
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,2018-06-20
...,...,...,...,...
10836,Sya9a Maroc - FR,FAMILY,4.5,2017-07-25
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,2018-07-06
10838,Parkinson Exercices FR,MEDICAL,NaN,2017-01-20
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,2015-01-19


Luego calculamos la diferencia de dias entre cada fecha de "Last Updated" y la fecha actual. Y eliminamos las columnas que ya no necesitamos antes de mergear

In [ ]:
fecha_actual = datetime.now()
apps3["Dias ultima actualizacion"] = (fecha_actual - apps3["Last Updated"]).dt.days
apps4 = apps3.drop(["Rating", "Last Updated"], axis = 1)
apps4

,App,Category,Dias ultima actualizacion
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,2100
1,Coloring book moana,ART_AND_DESIGN,2092
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,1894
3,Sketch - Draw & Paint,ART_AND_DESIGN,1948
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,1936
...,...,...,...
10836,Sya9a Maroc - FR,FAMILY,2266
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,1920
10838,Parkinson Exercices FR,MEDICAL,2452
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,3184


In [ ]:
appsreviews

,App,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,1.00,0.533333
1,10 Best Foods for You,0.25,0.288462
2,10 Best Foods for You,NaN,NaN
3,10 Best Foods for You,0.40,0.875000
4,10 Best Foods for You,1.00,0.300000
...,...,...,...
64290,Houzz Interior Design Ideas,NaN,NaN
64291,Houzz Interior Design Ideas,NaN,NaN
64292,Houzz Interior Design Ideas,NaN,NaN
64293,Houzz Interior Design Ideas,NaN,NaN


In [ ]:
appsreviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64295 entries, 0 to 64294
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   App                     64295 non-null  object 
 1   Sentiment_Polarity      37432 non-null  float64
 2   Sentiment_Subjectivity  37432 non-null  float64
dtypes: float64(2), object(1)
memory usage: 1.5+ MB


Vemos que hay nans en Sentiment_Polarity, podemos o reemplazar ese valor o eliminar aquellas filas. En mi criterio las eliminamos

In [ ]:
appsreviews["Sentiment_Polarity"].unique()

array([ 1.        ,  0.25      ,         nan, ..., -0.52857143,
       -0.37777778,  0.17333333])

Lo mismo con esta columna

In [ ]:
appsreviews["Sentiment_Subjectivity"].unique()

array([0.53333333, 0.28846154,        nan, ..., 0.51145833, 0.7172619 ,
       0.2594697 ])

Los eliminamos correctamente

In [ ]:
appsreviews1 = appsreviews.dropna(subset = ["Sentiment_Polarity", "Sentiment_Subjectivity"])
appsreviews1[["Sentiment_Polarity", "Sentiment_Subjectivity"]].isna().sum()

Sentiment_Polarity        0
Sentiment_Subjectivity    0
dtype: int64

Comprobamos que todas las polaridades esten en el rango posible

In [ ]:
((appsreviews1["Sentiment_Polarity"] > 1) | (appsreviews1["Sentiment_Polarity"] < -1)).sum()

0

In [ ]:
(appsreviews1["Sentiment_Subjectivity"] < 0).sum()

0

Combinamos ambos dataframes para que aquellas apps que esten en ambos figuren todos sus datos

In [ ]:
appstotal = pd.merge(apps4, appsreviews1, on = "App")
appstotal

,App,Category,Dias ultima actualizacion,Sentiment_Polarity,Sentiment_Subjectivity
0,Coloring book moana,ART_AND_DESIGN,2092,-0.250000,1.000000
1,Coloring book moana,ART_AND_DESIGN,2092,-0.725000,0.833333
2,Coloring book moana,ART_AND_DESIGN,2092,0.000000,0.000000
3,Coloring book moana,ART_AND_DESIGN,2092,0.500000,0.600000
4,Coloring book moana,ART_AND_DESIGN,2092,-0.800000,0.900000
...,...,...,...,...,...
35929,A+ Gallery - Photos & Videos,PHOTOGRAPHY,1889,0.475000,0.512500
35930,A+ Gallery - Photos & Videos,PHOTOGRAPHY,1889,0.566667,0.733333
35931,A+ Gallery - Photos & Videos,PHOTOGRAPHY,1889,0.200000,0.200000
35932,A+ Gallery - Photos & Videos,PHOTOGRAPHY,1889,0.000000,0.000000


No podemos hacer ahora directamente un groupby CATEGORY ya que el promedio de dias desde la ultima actualizacion estaria mal ya que para una misma
app esta repetido. Primero calculamos el promedio de antigüedad de cada categoria y lo guardamos en una columna

In [ ]:
aux = appstotal.drop_duplicates(subset = "App")
aux1 = aux.groupby("Category").mean()["Dias ultima actualizacion"]
aux1

<ipython-input-119-cb13affcd068>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  aux1 = aux.groupby("Category").mean()["Dias ultima actualizacion"]


Category
ART_AND_DESIGN         2025.100000
AUTO_AND_VEHICLES      1926.000000
BEAUTY                 2072.333333
BOOKS_AND_REFERENCE    2019.812500
BUSINESS               1918.550000
COMICS                 1945.500000
COMMUNICATION          2029.687500
DATING                 1956.545455
EDUCATION              2341.000000
ENTERTAINMENT          1958.560000
EVENTS                 1967.800000
FAMILY                 2036.037736
FINANCE                1915.743590
FOOD_AND_DRINK         1941.500000
GAME                   1957.086022
HEALTH_AND_FITNESS     1989.125000
HOUSE_AND_HOME         2089.933333
LIBRARIES_AND_DEMO     2731.222222
LIFESTYLE              2111.956522
MAPS_AND_NAVIGATION    1947.857143
MEDICAL                2085.636364
NEWS_AND_MAGAZINES     1970.875000
PARENTING              1937.750000
PERSONALIZATION        2166.441176
PHOTOGRAPHY            2012.774194
PRODUCTIVITY           2122.108108
SHOPPING               1914.400000
SOCIAL                 2168.666667
SPORTS     

Ahora regresamos a nuestro df appstotal y agrupamos por categoria para obtener su promedio de polaridad y de subjetividad

In [ ]:
appstotal1 = appstotal.groupby("Category").mean()[["Sentiment_Polarity", "Sentiment_Subjectivity"]]
appstotal1

<ipython-input-120-7412c7609d09>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  appstotal1 = appstotal.groupby("Category").mean()[["Sentiment_Polarity", "Sentiment_Subjectivity"]]


,Sentiment_Polarity,Sentiment_Subjectivity
Category,,
ART_AND_DESIGN,0.234735,0.537098
AUTO_AND_VEHICLES,0.348683,0.529918
BEAUTY,0.193360,0.494510
BOOKS_AND_REFERENCE,0.248422,0.476268
BUSINESS,0.232189,0.444924
COMICS,0.440031,0.558521
COMMUNICATION,0.177275,0.486109
DATING,0.178846,0.510476
EDUCATION,0.279332,0.493070


Y le agregamos a cada categoria su promedio de antigüedad antes calculado

In [ ]:
appstotal1["Promedio antigüedad en dias"] = aux1
appstotal1

,Sentiment_Polarity,Sentiment_Subjectivity,Promedio antigüedad en dias
Category,,,
ART_AND_DESIGN,0.234735,0.537098,2025.100000
AUTO_AND_VEHICLES,0.348683,0.529918,1926.000000
BEAUTY,0.193360,0.494510,2072.333333
BOOKS_AND_REFERENCE,0.248422,0.476268,2019.812500
BUSINESS,0.232189,0.444924,1918.550000
COMICS,0.440031,0.558521,1945.500000
COMMUNICATION,0.177275,0.486109,2029.687500
DATING,0.178846,0.510476,1956.545455
EDUCATION,0.279332,0.493070,2341.000000


Por ultimo calculamos la correlacion entre la antigüedad promedio con el promedio de polaridad y subjetividad

In [ ]:
correlacion = appstotal1.corr()
correlacion_fila = correlacion.loc["Promedio antigüedad en dias"]
respuesta = correlacion_fila[["Sentiment_Polarity", "Sentiment_Subjectivity"]]
respuesta

Sentiment_Polarity       -0.088744
Sentiment_Subjectivity    0.027165
Name: Promedio antigüedad en dias, dtype: float64